<a href="https://colab.research.google.com/github/yoonwanggyu/Self_Study/blob/main/%EC%98%A4%ED%94%84%EB%9D%BC%EC%9D%B8/%EC%9D%B4%EB%AF%B8%EC%A7%80_callback%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## EarlyStopping


- 첫번째 Callback 기능인 EarlyStopping 에 대해 살펴보자.
- 이 기능은 말그대로 모델 학습 시, 더 이상 모델 성능이 개선되지 않을 때, 모델  
  학습을 중단(조기 종료)하는 것이다.

- 그렇다면, 이 기능을 사용하기 위해서 필요한 세부 설정은 어떤 것이 있을지      
  생각해보자.

    - 모델 성능을 판단할 지표
    - 성능 개선 여부를 판단한 기준
    - 성능 개선이 이뤄지지 않을 때, 얼마나 지켜볼 것인지(인내할 것인지)
    - 조기 종료할 때, 종료 시 결과로 반환한 것인지, 최고 상태의 결과로 복원해서
      반환할 것인지

- tf.keras.callbacks.EarlyStopping(
     monitor='val_loss', -> 성능 판단 지표  
     min_delta=0, -> 개선이라 판단할 최소값    
     patience=0, -> 개선되지않을때 몇 epoch기다렸다 stop할것인지   
     restore_best_weights=False -> 최적weight값으로 복원할지  
     )

## ModelCheckpoint

- 무엇을 저장해야 할까?
    - 모델의 구조
    - 모델의 weights

In [ ]:
ckpt = tf.keras.callbacks.ModelCheckpoint(
     filepath=checkpoint_path, # 고유한 파일이름 부여 가능
     monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
     save_best_only=True, # True를 주면 monitor기준 최고상태 기록
     save_weights_only=True, # False면 모델 레이어 및 weight값 모두
     # save_freq=5 # 체크포인트 저장하는 주기 설정 가능 ('epoch'=에폭마다, 숫자=해당 배치마다)
     )

## ReduceLROnPlateau

- 모델이 일정기간(patience)동안 개선되지 않으면(monitor)
  Learning rate을 일정비율(factor)로 감소시킴으로 loss를 낮춤

In [ ]:
tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1, # new_lr = lr * factor.
    patience=10,
    min_delta=0.0001, # threshold for measuring the new optimum, to only focus on significant changes.
    cooldown=0, # number of epochs to wait before resuming normal operation after lr has been reduced.
    min_lr=0
)

- early stoping이랑 같이 쓸 때는 주의!!(reduce learning rate이 먼저 종료되도록
  설정)

## LearningRateScheduler

- 나만의 함수로 설정해서 learning rate을 줄이고 싶을 땐 learningratescheduler쓰자

- scheduler함수 설정을 바탕으로, epoch마다 learning rate을 변환

In [ ]:
def scheduler(epoch, lr):
   if epoch < 2:
     return lr
   else:
     return lr * 0.1

lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=0)

## 과제

- 아래 조건에 맞춰 콜백을 이용하여 cifar 10 데이터를 resnet 50으로 학습을 시켜봅시다.

    - early stopping : 50 에폭을 잡고, val loss 가 더이상 개선되지 않을 때 조기 종료 (세부 설정은 자율적으로 설정)
    - model save : weights만 저장 (세부 설정을 자율)
    - learning rate schedule : reduceLR을 사용하여, 성능개선이 되지 않을 때 10분의 1씩 감소(patience 가 early stopping patience보다 작게 잡을 것)
    - 학습이 완료되면, 저장된 weights를 불러와서 검증해봅시다.

In [1]:
# import tensorflow
import tensorflow as tf

(train_image,train_label),(test_image,test_label) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 3s 0us/step


In [2]:
train_image.shape,train_label.shape,test_image.shape,test_label.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [3]:
import keras
from keras.applications import *
from tensorflow.keras import layers

base_model = keras.applications.resnet50.ResNet50(weights='imagenet',
                                                  include_top = False,
                                                  input_shape = (32,32,3))

94765736/94765736 [==============================] - 0s 0us/step


In [4]:
len(base_model.layers)

175

In [5]:
base_model.trainable = True

for layer in base_model.layers[:-20]:
  layer.trainable = False

for layer in base_model.layers[-100:]:
  print(layer.name, layer.trainable)

base_model.output_shape

conv3_block4_2_bn False
conv3_block4_2_relu False
conv3_block4_3_conv False
conv3_block4_3_bn False
conv3_block4_add False
conv3_block4_out False
conv4_block1_1_conv False
conv4_block1_1_bn False
conv4_block1_1_relu False
conv4_block1_2_conv False
conv4_block1_2_bn False
conv4_block1_2_relu False
conv4_block1_0_conv False
conv4_block1_3_conv False
conv4_block1_0_bn False
conv4_block1_3_bn False
conv4_block1_add False
conv4_block1_out False
conv4_block2_1_conv False
conv4_block2_1_bn False
conv4_block2_1_relu False
conv4_block2_2_conv False
conv4_block2_2_bn False
conv4_block2_2_relu False
conv4_block2_3_conv False
conv4_block2_3_bn False
conv4_block2_add False
conv4_block2_out False
conv4_block3_1_conv False
conv4_block3_1_bn False
conv4_block3_1_relu False
conv4_block3_2_conv False
conv4_block3_2_bn False
conv4_block3_2_relu False
conv4_block3_3_conv False
conv4_block3_3_bn False
conv4_block3_add False
conv4_block3_out False
conv4_block4_1_conv False
conv4_block4_1_bn False
conv4_bloc

(None, 1, 1, 2048)

In [6]:
inputs = tf.keras.Input(shape=(32, 32, 3))

x = base_model(inputs, training=True)

x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x= tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

In [7]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
                  loss = 'sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [8]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                              patience = 15,
                                              verbose = 1,
                                              restore_best_weights = True)
check_point = tf.keras.callbacks.ModelCheckpoint(filepath = 'test_model.h5',
                                                 monitor = 'val_loss',
                                                 save_best_only = True,
                                                 save_weights_only = True)
reduceLr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss',
                                                patience = 5,
                                                factor = 0.1,   # 1/10 씩 감소시킴
                                                verbose = 1,
                                                min_lr = 0)

In [9]:
model.fit(train_image,train_label,
          validation_split = 0.2,
          epochs=50,
          callbacks=[early_stop,check_point,reduceLr],
          verbose = 1)

Epoch 1/50
1250/1250 [==============================] - 1041s 827ms/step - loss: 1.4536 - accuracy: 0.5016 - val_loss: 1.2080 - val_accuracy: 0.5815 - lr: 0.0010
Epoch 2/50
1250/1250 [==============================] - 1005s 804ms/step - loss: 1.1867 - accuracy: 0.5835 - val_loss: 1.1477 - val_accuracy: 0.5999 - lr: 0.0010
Epoch 3/50
 482/1250 [==========>...................] - ETA: 9:30 - loss: 1.0697 - accuracy: 0.6274

KeyboardInterrupt: 